1. Самостоятельно разобраться с тем, что такое tfidf (документация https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html и еще - https://scikit-learn.org/stable/modules/feature_extraction.html#text-feature-extraction)
2. Модифицировать код функции get_user_embedding таким образом, чтобы считалось не среднее (как в примере np.mean), а медиана. Применить такое преобразование к данным, обучить модель прогнозирования оттока и посчитать метрики качества и сохранить их: roc auc, precision/recall/f_score (для 3 последних - подобрать оптимальный порог с помощью precision_recall_curve, как это делалось на уроке)
3. Повторить п.2, но используя уже не медиану, а max
4. (опциональное, если очень хочется) Воспользовавшись полученными знаниями из п.1, повторить пункт 2, но уже взвешивая новости по tfidf (подсказка: нужно получить веса-коэффициенты для каждого документа. Не все документы одинаково информативны и несут какой-то положительный сигнал). Подсказка 2 - нужен именно idf, как вес.
5. Сформировать на выходе единую таблицу, сравнивающую качество 3 разных метода получения эмбедингов пользователей: mean, median, max, idf_mean по метрикам roc_auc, precision, recall, f_score
6. Сделать самостоятельные выводы и предположения о том, почему тот или ной способ оказался эффективнее остальных

## Сначала повторим действия с урока

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, precision_score, classification_report, precision_recall_curve, confusion_matrix
import itertools

import matplotlib.pyplot as plt

%matplotlib inline

Наши новости

In [ ]:
news = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/machine_learning_in_business/data/articles.csv")
print(news.shape)
news.head(3)

(27000, 2)


,doc_id,title
0,6,Заместитель председателяnправительства РФnСерг...
1,4896,Матч 1/16 финала Кубка России по футболу был п...
2,4897,Форвард «Авангарда» Томаш Заборский прокоммент...


Загрузим пользователей и списки последних прочитанных новостей

In [ ]:
users = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/machine_learning_in_business/data/users_articles.csv")
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


Итак, нам нужно получить векторные представления пользователей на основе прочитанным ими новостей и самих новостей

### 1. Получаем векторные представления новостей

In [ ]:
!pip install gensim

In [ ]:
#from gensim.test.utils import common_texts
from gensim.corpora.dictionary import Dictionary

In [ ]:
!pip install --upgrade razdel

Requirement already up-to-date: razdel in /usr/local/lib/python3.7/dist-packages (0.5.0)


In [ ]:
!pip install --upgrade nltk

Requirement already up-to-date: nltk in /usr/local/lib/python3.7/dist-packages (3.5)


In [ ]:
!pip install pymorphy2

In [ ]:
#предобработка текстов
import re
import numpy as np
import nltk
from nltk.corpus import stopwords
#from nltk.tokenize import word_tokenize

from razdel import tokenize # https://github.com/natasha/razdel
#!pip install razdel

import pymorphy2  # pip install pymorphy2

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Исследование по проблемам stop-листов: https://www.aclweb.org/anthology/W18-2502/

In [ ]:
stopword_ru = stopwords.words('russian')  # Убирают, например слова без смысловой нагрузки (местоимения, предлоги), мат и др.
len(stopword_ru)

morph = pymorphy2.MorphAnalyzer()  # Lemmatisation (Лемматизация)

Есть 2 метода решить проблему разных словоформ:
1. Stemming (Стемминг) - грубо говоря, обрезает концовки слов (подходит для английского языка, на русском плохо)
2. Lemmatisation (Лемматизация) - приводится словоформа на основании заранее написанных словарей (подходит для русского языка)

In [ ]:
with open(r"/content/drive/MyDrive/Colab Notebooks/machine_learning_in_business/data/stopwords.txt") as f:
    additional_stopwords = [w.strip() for w in f.readlines() if w]
stopword_ru += additional_stopwords
len(stopword_ru)

776

In [ ]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [ ]:
def clean_text(text):
    '''
    очистка текста
    
    на выходе очищеный текст (готовый к лемматизации)
    
    '''
    if not isinstance(text, str):
        text = str(text)
    
    text = text.lower()  # приводим к строчному виду
    text = text.strip('\n').strip('\r').strip('\t')
    text = re.sub("-\s\r\n\|-\s\r\n|\r\n", '', str(text))

    text = re.sub("[0-9]|[-—.,:;_%©«»?*!@#№$^•·&()]|[+=]|[[]|[]]|[/]|", '', text)
    text = re.sub(r"\r\n\t|\n|\\s|\r\t|\\n", ' ', text)
    text = re.sub(r'[\xad]|[\s+]', ' ', text.strip())
    
    #tokens = list(tokenize(text))
    #words = [_.text for _ in tokens]
    #words = [w for w in words if w not in stopword_ru]
    
    #return " ".join(words)
    return text

cache = {}

def lemmatization(text):
    '''
    лемматизация
        [0] если зашел тип не `str` делаем его `str`
        [1] токенизация предложения через razdel
        [2] проверка есть ли в начале слова '-'
        [3] проверка токена с одного символа
        [4] проверка есть ли данное слово в кэше
        [5] лемматизация слова
        [6] проверка на стоп-слова

    на выходе лист отлемматизированых токенов
    '''

    # [0]
    if not isinstance(text, str):
        text = str(text)
    
    # [1]
    tokens = list(tokenize(text))
    words = [_.text for _ in tokens] или считать по максимальным значениям, если важнее precision. Предположу, что preds_embeddings_mean сработало хуже у 

    words_lem = []
    for w in words:
        if w[0] == '-': # [2]
            w = w[1:]
        if len(w)>1: # [3]
            if w in cache: # [4]
                words_lem.append(cache[w])
            else: # [5]
                # morph.parse - выдает несколько нормальных форм в порядке убывания вероятности, но в этом кейсе мы берем только самую вероятную
                temp_cach = cache[w] = morph.parse(w)[0].normal_form
                words_lem.append(temp_cach)
    
    words_lem_without_stopwords=[i for i in words_lem if not i in stopword_ru] # [6]
    
    return words_lem_without_stopwords

In [ ]:
%%time
#Запускаем очистку текста. Будет долго...
news['title'] = news['title'].apply(lambda x: clean_text(x), 1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: FutureWarning: Possible nested set at position 39
  from ipykernel import kernelapp as app


CPU times: user 25.5 s, sys: 1.7 s, total: 27.2 s
Wall time: 27.3 s


In [ ]:
%%time
#Запускаем лемматизацию текста. Будет очень долго...
news['title'] = news['title'].apply(lambda x: lemmatization(x), 1)

CPU times: user 4min 33s, sys: 798 ms, total: 4min 34s
Wall time: 4min 34s


А теперь в 3 строчки обучим нашу модель

In [ ]:
#сформируем список наших текстов, разбив еще и на пробелы
texts = [t for t in news['title'].values]

# Create a corpus from a list of texts
common_dictionary = Dictionary(texts)
common_corpus = [common_dictionary.doc2bow(text) for text in texts]  # bow - bag of words

Все просто - это словарь наших слов

Запускаем обучение

In [ ]:
%%time
from gensim.models import LdaModel
# Train the model on the corpus
# 25 тематик задаём "руками" экспертно, принцип как в K-Means, алгоритм пытается разбить на указанное нами число тем
lda = LdaModel(common_corpus, num_topics=25, id2word=common_dictionary)#, passes=10)

CPU times: user 54.2 s, sys: 26.8 s, total: 1min 21s
Wall time: 51.7 s


Обучили модель. Теперь 2 вопроса:

1. как выглядят наши темы
2. как получить для документа вектор значений (вероятности принадлежности каждой теме)

In [ ]:
# Create a new corpus, made of previously unseen documents.
other_texts = [t for t in news['title'].iloc[:3]]
other_corpus = [common_dictionary.doc2bow(text) for text in other_texts]

unseen_doc = other_corpus[2]
print(other_texts[2])
lda[unseen_doc] 

['форвард', 'авангард', 'томаш', 'заборский', 'прокомментировать', 'игра', 'команда', 'матч', 'чемпионат', 'кхл', 'против', 'атланта', 'nnnn', 'плохой', 'матч', 'нижний', 'новгород', 'против', 'торпедо', 'настраиваться', 'первый', 'минута', 'включиться', 'заборский', 'получиться', 'забросить', 'быстрый', 'гол', 'задать', 'хороший', 'темп', 'поединок', 'играть', 'хороший', 'сторона', 'пять', 'очко', 'выезд', 'девять', 'хороший']


[(1, 0.17856407), (2, 0.071060926), (12, 0.4253282), (24, 0.304559)]

In [ ]:
x=lda.show_topics(num_topics=25, num_words=7,formatted=False)
topics_words = [(tp[0], [wd[0] for wd in tp[1]]) for tp in x]

#Below Code Prints Only Words 
for topic,words in topics_words:
    print("topic_{}: ".format(topic)+" ".join(words))
# Слова в ответе в порядке убывания их влияния на тему

topic_0: исследование сша учёный американский ракета санкция век
topic_1: всё рост очень земля большой уровень россия
topic_2: смерть препарат тело жизнь всё память болезнь
topic_3: египет оборудование nn живой ран продолжительность потепление
topic_4: ребёнок гражданин россия научный женщина животное исследователь
topic_5: советский снизить продукт общество наука геном московский
topic_6: писать испытание страдать nthe свет дом книга
topic_7: луна рак остров китай вода nn японский
topic_8: газ учёный исследователь космический нефть спутник новый
topic_9: военный россия наука министерство станция новый поток
topic_10: украина украинский экономика киев белоруссия белый россия
topic_11: конкурс наука супруг девочка уголь премия вуз
topic_12: космос восток планета расстояние канал мальчик небо
topic_13: млрд рубль банк цена рынок фонд млн
topic_14: форум nn место рейтинг мышь параметр определение
topic_15: nn глава совет эксперимент валюта решение организм
topic_16: доля млн университет т

Очень неплохо - большинство тем вполне можно описать о чем они

Давайте напишем функцию, которая будет нам возвращать векторное представление новости

In [ ]:
#text = news['title'].iloc[0]

# Теперь мы хотим, на основании разбиения по темам выше, каждую статью представлять в виде вектора, который содержит для каждой темы % близости к ней
def get_lda_vector(text):
    unseen_doc = common_dictionary.doc2bow(text)  # из новой статьи получили слова
    lda_tuple = lda[unseen_doc]  # Присвоили тематику
    not_null_topics = dict(zip([i[0] for i in lda_tuple], [i[1] for i in lda_tuple]))

    output_vector = []
    for i in range(25):
        if i not in not_null_topics:
            output_vector.append(0)
        else:
            output_vector.append(not_null_topics[i])
    return np.array(output_vector)

In [ ]:
topic_matrix = pd.DataFrame([get_lda_vector(text) for text in news['title'].values])
topic_matrix.columns = ['topic_{}'.format(i) for i in range(25)]
topic_matrix['doc_id'] = news['doc_id'].values
topic_matrix = topic_matrix[['doc_id']+['topic_{}'.format(i) for i in range(25)]]
topic_matrix.head(5)

,doc_id,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,6,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.059853,0.000000,0.662004,0.000000,0.111517,0.0,0.148688,0.000000,0.0,0.0,0.011215,0.0,0.0,0.000000
1,4896,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.048933,0.206972,0.000000,0.000000,0.000000,0.0,0.064026,0.000000,0.0,0.0,0.000000,0.0,0.0,0.659581
2,4897,0.0,0.163405,0.127505,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.397644,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.290958
3,4898,0.0,0.393815,0.191307,0.112697,0.066589,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.070579,0.000000,0.088075,0.000000,0.0,0.000000,0.068366,0.0,0.0,0.000000,0.0,0.0,0.000000
4,4899,0.0,0.072210,0.000000,0.000000,0.000000,0.097137,0.0,0.0,0.0,0.0,0.0,0.000000,0.091605,0.000000,0.000000,0.716345,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.0,0.000000


Прекрасно, мы получили вектора наших новостей! И даже умеем интерпретировать получившиеся темы.

Можно двигаться далее

### Следующий шаг - векторные представления пользователей

Далее мы хотим для каждого юзера взять векторы его статей где проставлены вероятностные распределения по темам и найти среднее по каждой теме. Это и будет эмбеддинг, характеризующий юзера.

In [ ]:
users.head(3)

,uid,articles
0,u105138,"[293672, 293328, 293001, 293622, 293126, 1852]"
1,u108690,"[3405, 1739, 2972, 1158, 1599, 322665]"
2,u108339,"[1845, 2009, 2356, 1424, 2939, 323389]"


In [ ]:
# topic_matrix переводим в векторную форму
doc_dict = dict(zip(topic_matrix['doc_id'].values, topic_matrix[['topic_{}'.format(i) for i in range(25)]].values))

## Модифицируем функцию get_user_embedding и сравним метрики

In [ ]:
def get_user_embedding_new(user_articles_list, method='mean'):
    """
    method: 'mean' / 'median' / 'max'
    """
    user_articles_list = eval(user_articles_list)
    user_vector = np.array([doc_dict[doc_id] for doc_id in user_articles_list])
    if method == 'mean':
        user_vector = np.mean(user_vector, axis=0)  # Вот тут находим среднее по темам для юзера
    elif method == 'median':
        user_vector = np.median(user_vector, axis=0)
    elif method == 'max':
        user_vector = np.max(user_vector, 0)  # идентично axis=0
    return user_vector

In [ ]:
user_articles_list = users['articles'].iloc[33]
get_user_embedding_new(user_articles_list, method='mean')
#get_user_embedding_new(user_articles_list, method='median')
#get_user_embedding_new(user_articles_list, method='max')

array([0.05655881, 0.05252899, 0.00217402, 0.        , 0.07903294,
       0.        , 0.        , 0.02688554, 0.01185732, 0.07967297,
       0.01197489, 0.00810417, 0.        , 0.0145253 , 0.        ,
       0.12341383, 0.        , 0.18974402, 0.        , 0.02021861,
       0.02447238, 0.03903681, 0.2306012 , 0.0124764 , 0.00331886])

Теперь получим эмбединги для всех пользователей и проверим их качество на конкретной downstream-задаче

In [ ]:
user_embeddings_mean = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_new(x, method='mean'), 1)])
user_embeddings_mean.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_mean['uid'] = users['uid'].values
user_embeddings_mean = user_embeddings_mean[['uid']+['topic_{}'.format(i) for i in range(25)]]
user_embeddings_mean.head(3)

,uid,topic_0,topic_1,topic_2,topic_3,topic_4,topic_5,topic_6,topic_7,topic_8,topic_9,topic_10,topic_11,topic_12,topic_13,topic_14,topic_15,topic_16,topic_17,topic_18,topic_19,topic_20,topic_21,topic_22,topic_23,topic_24
0,u105138,0.032645,0.036736,0.021159,0.004100,0.127754,0.054568,0.094175,0.000000,0.000000,0.035990,0.040073,0.040095,0.002765,0.000000,0.047739,0.104646,0.073463,0.118035,0.018667,0.000000,0.055156,0.015061,0.015198,0.023364,0.023328
1,u108690,0.025113,0.123114,0.002182,0.011190,0.074025,0.005112,0.002389,0.000000,0.000000,0.057041,0.126383,0.017151,0.005932,0.036217,0.006879,0.053808,0.002701,0.114957,0.056452,0.022348,0.038107,0.013341,0.141529,0.022408,0.027751
2,u108339,0.002503,0.027491,0.007229,0.008685,0.035024,0.000000,0.000000,0.054504,0.008956,0.115897,0.018095,0.000000,0.002677,0.022503,0.017765,0.080475,0.007531,0.198754,0.024490,0.013623,0.122559,0.030112,0.074670,0.051376,0.058700


In [ ]:
user_embeddings_median = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_new(x, method='median'), 1)])
user_embeddings_median.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_median['uid'] = users['uid'].values
user_embeddings_median = user_embeddings_median[['uid']+['topic_{}'.format(i) for i in range(25)]]

In [ ]:
user_embeddings_max = pd.DataFrame([i for i in users['articles'].apply(lambda x: get_user_embedding_new(x, method='max'), 1)])
user_embeddings_max.columns = ['topic_{}'.format(i) for i in range(25)]
user_embeddings_max['uid'] = users['uid'].values
user_embeddings_max = user_embeddings_max[['uid']+['topic_{}'.format(i) for i in range(25)]]

Датасет готов - можно попробовать обучить модель. Загрузим нашу разметку

In [ ]:
target = pd.read_csv(r"/content/drive/MyDrive/Colab Notebooks/machine_learning_in_business/data/users_churn.csv")
target.head(3)

,uid,churn
0,u107120,0
1,u102277,0
2,u102444,0


In [ ]:
def make_predictions(user_embeddings, target):
    X = pd.merge(user_embeddings, target, 'left')
    X_train, X_test, y_train, y_test = train_test_split(X[['topic_{}'.format(i) for i in range(25)]], 
                                                    X['churn'], random_state=0)
    logreg = LogisticRegression()
    #обучим
    logreg.fit(X_train, y_train)
    #наши прогнозы для тестовой выборки
    preds = logreg.predict_proba(X_test)[:, 1]
    return preds

In [ ]:
preds_embeddings_mean = make_predictions(user_embeddings_mean, target)
preds_embeddings_median = make_predictions(user_embeddings_median, target)
preds_embeddings_max = make_predictions(user_embeddings_max, target)
preds_embeddings_mean[:10]

array([0.2138205 , 0.02330607, 0.3836392 , 0.42299591, 0.04971261,
       0.03834308, 0.15292561, 0.05150341, 0.05278461, 0.13061209])

### Рассчитаем Precision, Recall, F_score

In [ ]:
def metrics_calculation(y_test, preds):

    roc_auc = roc_auc_score(y_test, preds)
    precision, recall, thresholds = precision_recall_curve(y_test, preds)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    return thresholds[ix], fscore[ix], precision[ix], recall[ix], roc_auc

In [ ]:
all_preds = [preds_embeddings_mean, preds_embeddings_median, preds_embeddings_max]
metrics = []
for preds in all_preds:
    best_threshold, fscore, precision, recall, roc_auc = metrics_calculation(y_test, preds)
    metrics.append([fscore, precision, recall, roc_auc])

In [ ]:
metrics_comparison_df = pd.DataFrame(np.array(metrics),
                   columns=['fscore', 'precision', 'recall', 'roc_auc'],
                   index=['preds_embeddings_mean', 'preds_embeddings_median', 'preds_embeddings_max'])
metrics_comparison_df

,fscore,precision,recall,roc_auc
preds_embeddings_mean,0.692857,0.615873,0.791837,0.950095
preds_embeddings_median,0.757282,0.722222,0.795918,0.967580
preds_embeddings_max,0.762475,0.746094,0.779592,0.970698


В данном кейсе можно останавливаться на эмбеддингах по медиане, если для нас важнее recall или считать по максимальным значениям, если важнее precision. Предположу, что preds_embeddings_mean сработало хуже у меня, т.к. я на применял TF-IDF и выбросы испортили результат, т.к. среднее чувствительно к выбросам.